In [1]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 0.1.0 ──

✔ broom     0.5.6      ✔ rsample   0.0.5 
✔ dials     0.0.6      ✔ tune      0.1.0 
✔ infer     0.5.1      ✔ workflows 0.1.1 
✔ parsnip   0.1.0      ✔ yardstick 0.0.6 
✔ recipes   0.1.12     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ dials::margin()   masks ggplot2::margin()
✖ yardstick::spec() masks readr::spec()
✖ recip

## Covariates, phenotypes, population labels

In [2]:
# Load covariates (age, sex, interactions, PCs)
covariates <- read_tsv('../data/ukb_merged/covar_all_samples.covar', 
                       col_types = cols(.default = col_double())) %>%
    select(-'#FID')

# Load phenotypes
phenotypes <- read_delim('../data/phenotypes/full_phenotypes.pheno', 
                         delim = ' ', trim_ws = T, col_types = cols(.default = col_double())) %>%
    select(-'#FID')

In [3]:
test_pop_files <- c('../data/ukb_populations/AFR_all.txt', '../data/ukb_populations/AMR_all.txt',
                    '../data/ukb_populations/EAS_all.txt', '../data/ukb_populations/SAS_all.txt', 
                    '../data/ukb_populations/EUR_test.txt')

test_individuals <- test_pop_files %>%
    purrr::map(function(path) read_delim(path, delim = ' ', trim_ws = T, 
                                         col_types = cols(.default = col_double()))) %>%
    bind_rows()
               
test_individuals %>% write_tsv('../data/ukb_populations/test_all.tsv')

test_iids <- test_individuals %>% select(IID)

## Distance metrics

In [4]:
# Pairwise Fst between individuals in the test population and all individuals from the train population
fst_df <- read_tsv('../data/ukb_populations/final_fst.tsv', 
                   col_types = cols(.default = col_double())) %>%
    select(IID, fst = weighted_fst)

In [29]:
# Compute 1 and 2 norm PC distances
train_iids <- read_delim('../data/ukb_populations/EUR_all.txt', delim = ' ', trim_ws = T,
                         col_types = cols_only('IID' = col_double())) %>%
    anti_join(test_iids, by = 'IID')

set.seed(100)

train_pcs <- covariates %>% 
    select(IID, starts_with("PC")) %>%
    drop_na() %>%
    inner_join(train_iids %>% sample_n(10000), by = 'IID') %>%
    select(-IID)

# Compute the mean distance between X and Y
compute_mean_dist <- function(X, Y, metric, n_cols = 20) {
    rdist::cdist(
        X %>% magrittr::extract(1:n_cols), 
        Y %>% magrittr::extract(1:n_cols), 
        metric = metric
    ) %>% mean
}

test_mean_distances <- covariates %>% 
    select(IID, starts_with("PC")) %>%
    drop_na() %>%
    inner_join(test_iids, by = 'IID') %>%
    nest(data = -IID) %>%
    mutate(
        manhattan_20 = data %>% map_dbl(compute_mean_dist, Y = train_pcs, metric = 'manhattan', n_cols = 20),
        euclidean_20 = data %>% map_dbl(compute_mean_dist, Y = train_pcs, metric = 'euclidean', n_cols = 20),        
    ) %>% 
    select(-data)

test_mean_distances %>% head(2)

In [30]:
ibs_ibd_df <- read_tsv('../data/ukb_distances/ibd_complete.tsv', 
                       col_types = cols(.default = col_double())) %>%
    mutate(
        ibs = DST / train_iid,
        ibd = PI_HAT / train_iid,
    ) %>%
    select(IID = test_iid, ibs, ibd)

In [ ]:
distance_df <- fst_df %>%
    full_join(test_mean_distances, by = "IID") %>%
    full_join(ibs_ibd_df, by = 'IID') %>%
    pivot_longer(-IID, names_to = 'distance_metric', values_to = 'distance')

distance_df %>% head(2)

In [33]:
distance_df <- distance_df %>%
    filter(!distance_metric %>% str_detect('5'))

## PRS

In [34]:
# Create a single file for all polygenic score predictions (for test only)
prs_files <- list.files(path = '../data/prs/', pattern = '[A-Za-z]_[0-4]_scores.sscore', full.names = T)

format_prs_file <- function(file_path, iids_to_keep) {
    # Get relevant information from the file name
    trait_threshold <- file_path %>% str_extract('(?<=//)[A-Za-z]+_[0-4](?=_scores.sscore)')
    rename_cols <- list()
    rename_cols[[trait_threshold]] = 'SCORE1_AVG'
        
    col_types <- cols_only('IID' = col_double(), 'SCORE1_AVG' = col_double())
    read_tsv(file_path, col_types = col_types) %>%
        inner_join(iids_to_keep, by = 'IID') %>%
        rename(!!!rename_cols)
}

prs_test_individuals_df <- prs_files %>%
    map(format_prs_file, iids_to_keep = test_iids) %>%
    reduce(inner_join, by = "IID")

prs_test_individuals_df %>% nrow
prs_test_individuals_df %>% write_tsv('../data/prs/combined_test_prs.tsv')

In [ ]:
# Pivot PRS and phenotypes to long format
tidy_prs <- prs_test_individuals_df %>% 
    pivot_longer(-IID, names_to = 'phenotype_threshold', values_to = 'prs') %>%
    separate(phenotype_threshold, c('phenotype', 'threshold'))

tidy_phenotypes <- phenotypes %>%
    pivot_longer(-IID, names_to = 'phenotype', values_to = 'phenotype_value')

# Combine covariates, phenotypes, and PRS into a single table
full_test_df <- test_iids %>%
    inner_join(covariates, by = 'IID') %>%
    inner_join(tidy_phenotypes, by = 'IID') %>%
    inner_join(tidy_prs, by = c('IID', 'phenotype'))

full_test_df %>% nrow
full_test_df %>% head(2)

## Evaluation

In [36]:
compute_incremental_r2 <- function(df) {
    nested_formula = as.formula(phenotype_value ~ sex_covar + age + age_sq + age_sex + age_sq_sex + 
                                PC1_AVG + PC2_AVG + PC3_AVG + PC4_AVG + PC5_AVG + PC6_AVG + PC7_AVG + 
                                PC8_AVG + PC9_AVG + PC10_AVG)
    
    nested = lm(nested_formula, data = as_tibble(df))
    full = lm(nested_formula %>% update(~ . + prs), data = as_tibble(df))
    
    # Return incremental R^2
    summary(full)$adj.r.squared - summary(nested)$adj.r.squared
}

bootstrapped_evaluation <- function(data, times = 10) {
    data %>%
        bootstraps(times = times) %>%
        mutate(bootstrap_incremental_r2 = splits %>% map_dbl(compute_incremental_r2)) %>%
        select(-splits)
}

In [37]:
bootstrapped_df <- full_test_df %>% 
    inner_join(distance_df, by = 'IID') %>%
    group_by(distance_metric) %>%
    mutate(distance_group = distance %>% ntile(5)) %>%
    ungroup %>%
    group_by(distance_metric, distance_group) %>%
    mutate(distance_group_mean = mean(distance)) %>%
    ungroup %>%
    nest(data = c(-phenotype, -threshold, -distance_metric, -distance_group, -distance_group_mean)) %>%
    mutate(
        incremental_r2 = data %>% map_dbl(compute_incremental_r2),
        bootstrap_incremental_r2 = data %>% map(bootstrapped_evaluation, times = 10)
    ) %>%
    unnest(bootstrap_incremental_r2) %>%
    select(-data, bootstrap_iter = id) %>%
    # Add bootstrap summary information (while keeping the specific values)
    group_by_at(vars(-bootstrap_iter, -bootstrap_incremental_r2)) %>%
    mutate(
        conf.lower = quantile(bootstrap_incremental_r2, probs = 0.025),
        conf.upper = quantile(bootstrap_incremental_r2, probs = 0.975)
    ) %>%
    ungroup

bootstrapped_df %>% write_tsv('../data/fst/multi_metric_bootstrap_evaluation.tsv')

bootstrapped_df %>% head(2)

phenotype,threshold,distance_metric,distance_group,distance_group_mean,incremental_r2,bootstrap_iter,bootstrap_incremental_r2,conf.lower,conf.upper
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
Basophil,0,fst,5,0.1294594,0.001729775,Bootstrap01,0.003045868,0.0006508898,0.003068478
Basophil,0,fst,5,0.1294594,0.001729775,Bootstrap02,0.001498795,0.0006508898,0.003068478
